In [1]:
import pandas as pd
url = 'https://barttorvik.com/trank.php?year={yr}&sort=&top=0&conlimit=All&venue=All&type=R#'
dfs = []
for x in range(2017,2025):
    print(x)
    print(url.format(yr=x))
    df = pd.read_html(url.format(yr=x), header= 1)[0]
    df['Year'] = x
    dfs.append(df)
finaldf = pd.concat(dfs,ignore_index= True)

2017
https://barttorvik.com/trank.php?year=2017&sort=&top=0&conlimit=All&venue=All&type=R#
2018
https://barttorvik.com/trank.php?year=2018&sort=&top=0&conlimit=All&venue=All&type=R#
2019
https://barttorvik.com/trank.php?year=2019&sort=&top=0&conlimit=All&venue=All&type=R#
2020
https://barttorvik.com/trank.php?year=2020&sort=&top=0&conlimit=All&venue=All&type=R#
2021
https://barttorvik.com/trank.php?year=2021&sort=&top=0&conlimit=All&venue=All&type=R#
2022
https://barttorvik.com/trank.php?year=2022&sort=&top=0&conlimit=All&venue=All&type=R#
2023
https://barttorvik.com/trank.php?year=2023&sort=&top=0&conlimit=All&venue=All&type=R#
2024
https://barttorvik.com/trank.php?year=2024&sort=&top=0&conlimit=All&venue=All&type=R#


In [2]:
#Clean up the dataset
finaldf[['Rec', 'Conf Record']] = finaldf['Rec'].str.split(' ', expand=True)
finaldf[['AdjOE', 'AdjOE Rank']] = finaldf['AdjOE'].str.split(' ', expand=True)
finaldf[['AdjDE', 'AdjDE Rank']] = finaldf['AdjDE'].str.split(' ', expand=True)
finaldf[['Barthag', 'Barthag Rank']] = finaldf['Barthag'].str.split(' ', expand=True)
finaldf[['EFG%', 'EFG% Rank']] = finaldf['EFG%'].str.split(' ', expand=True)
finaldf[['EFGD%', 'EFGD% Rank']] = finaldf['EFGD%'].str.split(' ', expand=True)
finaldf[['TOR', 'TOR Rank']] = finaldf['TOR'].str.split(' ', expand=True)
finaldf[['ORB', 'ORB Rank']] = finaldf['ORB'].str.split(' ', expand=True)
finaldf[['DRB', 'DRB Rank']] = finaldf['DRB'].str.split(' ', expand=True)
finaldf[['FTR', 'FTR Rank']] = finaldf['FTR'].str.split(' ', expand=True)
finaldf[['FTRD', 'FTRD Rank']] = finaldf['FTRD'].str.split(' ', expand=True)
finaldf[['2P%', '2P% Rank']] = finaldf['2P%'].str.split(' ', expand=True)
finaldf[['2P%D', '2P%D Rank']] = finaldf['2P%D'].str.split(' ', expand=True)
finaldf[['3P%', '3P% Rank']] = finaldf['3P%'].str.split(' ', expand=True)
finaldf[['3P%D', '3P%D Rank']] = finaldf['3P%D'].str.split(' ', expand=True)
finaldf[['Adj T.', 'Adj T. Rank']] = finaldf['Adj T.'].str.split(' ', expand=True)
finaldf = finaldf[finaldf['Team']!= "Team"]
finaldf = finaldf.apply(pd.to_numeric, errors='ignore')



C:\Users\Manas\AppData\Local\Temp\ipykernel_23672\2548163801.py:19: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  finaldf = finaldf.apply(pd.to_numeric, errors='ignore')


In [3]:
import re
def extract_tournament_result(input_string):
    if "seed" in input_string:
        parts = input_string.split("seed,")
        if len(parts) > 1:
            tounament_result = parts[1].strip()
            return tounament_result
    return None

def extract_seed(input_string):
    match = re.search(r'(...)(?=seed)',input_string)
    if match:
        return match.group(1)
    return None

def extract_team_name(input_string):
    # Use regular expression to find three characters before "seed"
    match = re.search(r'^(.*?)...(?=seed)', input_string)
    if match:
        # Extract the matched substring
        return match.group(1)
    return input_string

#Adding some labeling columns for analysis later
def get_tournament_games_won (input_string):
    
    if(input_string== "R68" or input_string == "R64"):
            return 0
    else:
          if(input_string == "R32"):
                return 1
          else:
                if(input_string == "Sweet Sixteen"):
                      return 2
                else:
                      if(input_string == "Elite Eight"):
                            return 3
                      else:
                            if(input_string == "Final Four"):
                                  return 4
                            else:
                                  if(input_string == "Finals"):
                                        return 5
                                  else:
                                        if(input_string =="CHAMPS"):
                                              return 6
                                        else:
                                            return 0
                                            

In [4]:
finaldf['Tournament_Result'] = finaldf['Team'].apply(extract_tournament_result)
finaldf['Tournament_Seed'] = finaldf['Team'].apply(extract_seed)
finaldf['Tournament_Games_Won'] = finaldf['Tournament_Result'].apply(get_tournament_games_won)

In [5]:
finaldf = finaldf.apply(pd.to_numeric, errors='ignore')

C:\Users\Manas\AppData\Local\Temp\ipykernel_23672\3362141772.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  finaldf = finaldf.apply(pd.to_numeric, errors='ignore')


In [6]:
import numpy as np
finaldf['is_underperformer'] = np.where((finaldf['Tournament_Games_Won'] < 4) & (finaldf['Tournament_Seed'] == 1),1,
                                       np.where((finaldf['Tournament_Games_Won'] < 3) & (finaldf['Tournament_Seed'] ==2),1,
                                                np.where((finaldf['Tournament_Games_Won'] < 2) & (finaldf['Tournament_Seed'] ==3),1,
                                                         np.where((finaldf['Tournament_Games_Won']<2) & (finaldf['Tournament_Seed'] ==4),1,
                                                                  np.where((finaldf['Tournament_Games_Won'] < 1) & (finaldf['Tournament_Seed'] <=9),1,0)))))
finaldf['is_overperformer'] = np.where((finaldf['Tournament_Games_Won'] > 4) & (finaldf['Tournament_Seed'] == 1),1,
                                       np.where((finaldf['Tournament_Games_Won'] > 3) & (finaldf['Tournament_Seed'] ==2),1,
                                                np.where((finaldf['Tournament_Games_Won'] >2) & (finaldf['Tournament_Seed'] ==3),1,
                                                         np.where((finaldf['Tournament_Games_Won']> 2) & (finaldf['Tournament_Seed'] ==4),1,
                                                                  np.where((finaldf['Tournament_Games_Won'] > 1) & (finaldf['Tournament_Seed'] <= 9),1,
                                                                           np.where((finaldf['Tournament_Games_Won'] > 0) & (finaldf['Tournament_Seed'] >= 10),1,0))))))
finaldf['is_expectedperformer'] = np.where((finaldf['Tournament_Games_Won'] == 4) & (finaldf['Tournament_Seed'] == 1),1,
                                       np.where((finaldf['Tournament_Games_Won'] == 3) & (finaldf['Tournament_Seed'] ==2),1,
                                                np.where((finaldf['Tournament_Games_Won'] ==2) & (finaldf['Tournament_Seed'] ==3),1,
                                                         np.where((finaldf['Tournament_Games_Won']== 2) & (finaldf['Tournament_Seed'] ==4),1,
                                                                  np.where((finaldf['Tournament_Games_Won'] == 1) & (finaldf['Tournament_Seed'] <= 9),1,
                                                                           np.where((finaldf['Tournament_Games_Won'] == 0) & (finaldf['Tournament_Seed'] >= 10),1,0))))))

In [7]:
finaldf['Team'] = finaldf['Team'].apply(extract_team_name)

In [18]:
TournamentTeams = finaldf.dropna(subset=['Tournament_Result']).copy()
TournamentTeams = TournamentTeams.drop(TournamentTeams[TournamentTeams['Year'] == 2024].index)
print(TournamentTeams.head())

   Rk             Team Conf   G    Rec  AdjOE  AdjDE  Barthag  EFG%  EFGD%  \
0   1         Gonzaga   WCC  33   32–1  119.8   88.0   0.9720  58.0   41.6   
1   2       Villanova    BE  34   31–3  122.2   92.2   0.9622  57.9   48.1   
2   3        Virginia   ACC  32  22–10  114.1   86.7   0.9591  52.9   45.7   
3   4        Kentucky   SEC  34   29–5  119.2   91.5   0.9545  53.2   47.4   
4   5  North Carolina   ACC  33   26–7  121.0   93.1   0.9531  52.3   48.5   

   ...  2P%D Rank 3P% Rank  3P%D Rank  Adj T. Rank  Tournament_Result  \
0  ...          2       43          7           65             Finals   
1  ...        155       78         25          315                R32   
2  ...         28       19         22          351                R32   
3  ...        122      156         13           14        Elite Eight   
4  ...         71       88        117           38             CHAMPS   

   Tournament_Seed  Tournament_Games_Won  is_underperformer  is_overperformer  \
0          

In [19]:
modelfeatures = TournamentTeams[['Tournament_Games_Won','AdjOE','AdjDE','TOR', 'ORB','DRB', 'Adj T.']].copy()
modelfeatures.corr()


,Tournament_Games_Won,AdjOE,AdjDE,TOR,ORB,DRB,Adj T.
Tournament_Games_Won,1.000000,0.472951,-0.388413,-0.145804,0.207708,-0.008946,0.020362
AdjOE,0.472951,1.000000,-0.303562,-0.489143,0.189684,-0.047792,0.131177
AdjDE,-0.388413,-0.303562,1.000000,-0.007688,-0.266687,0.172889,0.184447
TOR,-0.145804,-0.489143,-0.007688,1.000000,0.352989,0.132198,0.049871
ORB,0.207708,0.189684,-0.266687,0.352989,1.000000,0.098562,0.064614
DRB,-0.008946,-0.047792,0.172889,0.132198,0.098562,1.000000,0.232095
Adj T.,0.020362,0.131177,0.184447,0.049871,0.064614,0.232095,1.000000


In [20]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

#find design matrix for regression model using 'rating' as response variable 
y, X = dmatrices('Tournament_Games_Won ~ AdjOE+AdjDE+ORB+DRB+TOR', data=modelfeatures, return_type='dataframe')

#create DataFrame to hold VIF values
vif_df = pd.DataFrame()
vif_df['variable'] = X.columns 

#calculate VIF for each predictor variable 
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#view VIF for each predictor variable 
print(vif_df)

    variable          VIF
0  Intercept  1636.117642
1      AdjOE     1.764471
2      AdjDE     1.219507
3        ORB     1.466041
4        DRB     1.065982
5        TOR     1.834527


In [30]:
# Step 3: Separating the data into features and labels
X = TournamentTeams[['AdjOE','AdjDE','TOR', 'ORB','DRB']]
y = TournamentTeams['Tournament_Games_Won']

print(X)
print(y)

      AdjOE  AdjDE   TOR   ORB   DRB
0     119.8   88.0  15.9  29.9  25.4
1     122.2   92.2  17.1  29.6  27.6
2     114.1   86.7  15.6  28.7  24.2
3     119.2   91.5  16.0  34.0  27.2
4     121.0   93.1  16.4  42.0  25.1
...     ...    ...   ...   ...   ...
2394  104.4  106.2  18.0  32.5  27.9
2436  100.9  105.7  22.8  33.8  31.6
2473   98.7  106.9  17.3  25.3  27.9
2497   94.8  104.3  20.3  29.3  27.7
2515  104.6  117.0  16.3  31.4  28.9

[408 rows x 5 columns]
0       5
1       1
2       1
3       3
4       6
       ..
2394    0
2436    0
2473    0
2497    0
2515    1
Name: Tournament_Games_Won, Length: 408, dtype: int64


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                X, y, 
                test_size=0.3, 
                random_state=10, 
                shuffle=True
                )

In [32]:
from sklearn import metrics
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
# Step 6: Validation
# Evaluating the trained model on training data
y_prediction = model.predict(X_train)
print("MAE on train data= " , metrics.mean_absolute_error(y_train, y_prediction))
# Evaluating the trained model on test data
y_prediction = model.predict(X_test)
print("MAE on test data = " , metrics.mean_absolute_error(y_test, y_prediction))

print(model.coef_)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

MAE on train data=  0.7728730503608314
MAE on test data =  0.8898475108527467
[ 0.07377148 -0.06973913 -0.01767889  0.04239622  0.03052692]
0.3333272934154672
0.19835336675816995


In [33]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()
lr.fit(X_train,y_train)
 # Train multinomial logistic regression model
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train, y_train)
    
print("Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_train, lr.predict(X_train)))
print("Logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, lr.predict(X_test)))
    
print("Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_train, mul_lr.predict(X_train)))
print("Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, mul_lr.predict(X_test)))

c:\Users\Manas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic regression Train Accuracy ::  0.5859649122807018
Logistic regression Test Accuracy ::  0.5284552845528455
Multinomial Logistic regression Train Accuracy ::  0.5929824561403508
Multinomial Logistic regression Test Accuracy ::  0.5203252032520326


In [25]:
CYData = finaldf[finaldf['Year'] == 2024].copy()
print(CYData)

       Rk                    Team  Conf   G   Rec  AdjOE  AdjDE  Barthag  \
2574    1                Houston    B12  34  30–4  119.0   85.0   0.9796   
2575    2            Connecticut     BE  34  31–3  126.7   94.3   0.9677   
2576    3                 Purdue    B10  33  29–4  126.2   94.0   0.9671   
2577    4                 Auburn    SEC  34  27–7  120.7   92.2   0.9567   
2578    5               Iowa St.    B12  34  27–7  113.9   87.3   0.9551   
...   ...                     ...   ...  ..   ...    ...    ...      ...   
2944  358               Stonehill   NEC  30  3–27   90.4  113.9   0.0658   
2945  359          St. Francis PA   NEC  28  6–22   93.2  117.5   0.0649   
2946  360                   IUPUI  Horz  29  3–26   92.5  117.0   0.0630   
2947  361              Coppin St.  MEAC  29  2–27   84.8  110.2   0.0467   
2948  362  Mississippi Valley St.  SWAC  31  1–30   85.2  116.9   0.0258   

      EFG%  EFGD%  ...  2P%D Rank 3P% Rank  3P%D Rank  Adj T. Rank  \
2574  49.7   44.0

In [26]:
X2 = CYData[['AdjOE','AdjDE','TOR', 'ORB','DRB', 'Adj T.']]
yhats_mnlog = mul_lr.predict(X2)
CYData['Predicted Games Won Logistic'] = yhats_mnlog
yhats_lin = model.predict(X2)
CYData['Predicted Games Won Linear'] = yhats_lin

In [27]:
CYData.to_excel('Final Model.xlsx')

In [28]:
yhats_mnlog_historical = mul_lr.predict(X)
TournamentTeams['Predicted Games Logistic'] = yhats_mnlog_historical
yhats_lin_historical = model.predict(X)
TournamentTeams['Predicted Games Won Linear'] = yhats_lin_historical

In [29]:
TournamentTeams.to_excel('Final Model Historical.xlsx')